import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pandas as pd

# Load the training and testing data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Lab/MNIST/MNIST_submit/MNIST_LLM/train.csv")
# test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Lab/MNIST/MNIST_submit/MNIST_LLM/test.csv")

label = train_data['label'].values
train_data.drop(['label'], axis = 1, inplace = True)
train_data = train_data/255.0

test_data = test_data.values
test_data = test_data / 255.0


# Print the shape of the data
print("Training data shape:", train_data.shape)
print("Training labels shape:", label.shape)
print("Testing data shape:", test_data.shape)

Training data shape: (42000, 784)
Training labels shape: (42000,)
Testing data shape: (28000, 784)


In [2]:
# Random Forest, Xgboost, Voting Ensemble
'''
I've already completed feature engineering and conducted data preprocessing of MNIST dataset. 
In my opinion, hyperparameter tuning is the most effective method for improving performance. 
Find tools and Python code for hyperparameter tuning of Random Forest, XGBoost, and a voting ensemble of Random Forest and XGBoost.
'''
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Load your data
X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.2, random_state=42)

In [3]:
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score

#radom forest
rmf = RandomForestClassifier()
rmf.fit(X_train, y_train)
Y_pred = rmf.predict(X_test)
rmf.score(X_test, y_test)
acc_rmf = round(rmf.score(X_test, y_test) * 100, 2)
acc_rmf


KeyboardInterrupt



In [5]:
#xgboost
xgb = xgboost.XGBClassifier(tree_method='gpu_hist', gpu_id=0)
xgb.fit(X_train, y_train)
Y_pred = xgb.predict(X_test)
xgb.score(X_test, y_test)
acc_xgb = round(xgb.score(X_test, y_test) * 100, 2)
acc_xgb

97.39

In [6]:
#ensemble model -> randomforest, xgboost 모두 100퍼센트 나옴
from sklearn.ensemble import VotingClassifier

ves = VotingClassifier(estimators=[("Random Forest",rmf),
                                               ("XGBoost",xgb)],
                                  voting = 'soft')
ves.fit(X_train,y_train)
Y_pred_ves = ves.predict(X_test)
acc_ves = round(ves.score(X_test,y_test) * 100, 2)
acc_ves

97.43

In [11]:
sample = pd.read_csv('./sample_submission.csv')
ImageId = sample['ImageId']

Y_pred_rmf = rmf.predict(test_data)
submission = pd.DataFrame({
    "ImageId" : ImageId,
    "Label" : Y_pred_rmf
})
submission.to_csv('submission_rmf.csv', index = False)

Y_pred_xgb = xgb.predict(test_data)
submission = pd.DataFrame({
    "ImageId" : ImageId,
    "Label" : Y_pred_xgb
})
submission.to_csv('submission_xgb.csv', index = False)

Y_pred_ves = ves.predict(test_data)
submission = pd.DataFrame({
    "ImageId" : ImageId,
    "Label" : Y_pred_ves
})
submission.to_csv('submission_ves.csv', index = False)

### HyperTuning

In [9]:
# Hyperparameter tuning for Random Forest
rf = RandomForestClassifier()
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}
rf_grid = GridSearchCV(rf, rf_param_grid, cv=5, scoring='accuracy')
rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_

/home/dummy/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "/home/dummy/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dummy/.local/lib/python3.10/site-packages/sklearn/base.py", line 1344, in wrapper
    estimator._validate_params()
  File "/home/dummy/.local/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/dummy/.local/

In [10]:
print(rf_grid.best_params_)

{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [11]:
# Hyperparameter tuning for XGBoost
# xgb_model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0)
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9]
}
xgb_grid = GridSearchCV(xgb, xgb_param_grid, cv=5, scoring='accuracy')
xgb_grid.fit(X_train, y_train)
best_xgb = xgb_grid.best_estimator_

In [12]:
xgb_grid.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.3,
 'max_depth': 5,
 'n_estimators': 300,
 'subsample': 0.9}

In [13]:
# Create a voting ensemble
voting_ensemble = VotingClassifier(
    estimators=[('rf', best_rf), ('xgb', best_xgb)],
    voting='soft'  # Choose 'hard' or 'soft' voting strategy
)
voting_ensemble.fit(X_train, y_train)

# Evaluate the models
y_pred_rf = best_rf.predict(X_test)
y_pred_xgb = best_xgb.predict(X_test)
y_pred_ensemble = voting_ensemble.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Voting Ensemble Accuracy:", accuracy_score(y_test, y_pred_ensemble))

Random Forest Accuracy: 0.964047619047619
XGBoost Accuracy: 0.9761904761904762
Voting Ensemble Accuracy: 0.9758333333333333


In [16]:
import joblib

joblib.dump(best_rf,'./gpt_rf_HP.pkl')
joblib.dump(best_xgb,'./gpt_xgb_HP.pkl')
joblib.dump(voting_ensemble,'./gpt_vt_HP.pkl')

['./gpt_vt_HP.pkl']

In [ ]:
sample = pd.read_csv('./sample_submission.csv')
ImageId = sample['ImageId']

Y_pred_rmf = best_rf.predict(df_test)
submission = pd.DataFrame({
    "ImageId" : ImageId,
    "Label" : Y_pred_rmf
})
submission.to_csv('submission_rmf.csv', index = False)

Y_pred_xgb = best_xgb.predict(df_test)
submission = pd.DataFrame({
    "ImageId" : ImageId,
    "Label" : Y_pred_xgb
})
submission.to_csv('submission_xgb.csv', index = False)

Y_pred_ves = voting_ensemble.predict(df_test)
submission = pd.DataFrame({
    "ImageId" : ImageId,
    "Label" : Y_pred_ves
})
submission.to_csv('submission_ves.csv', index = False)

### Build CNN model

In [3]:
# test_data 가 빈 데이터로 변하는 현상 확인함.
# 보류
'''
import pandas as pd

# Load the training and testing data
# train_data = pd.read_csv("train.csv")
# test_data = pd.read_csv("test.csv")

# Concatenate train and test datasets
combined_data = pd.concat([train_data, test_data], axis=0)

# Find missing values
missing_values = combined_data.isnull().sum()
print("Missing values:")
print(missing_values)

# Handle missing values (for example, dropping rows with missing values)
combined_data.dropna(inplace=True)

# Separate back into train and test datasets
train_data = combined_data.iloc[:len(train_data)]
test_data = combined_data.iloc[len(train_data):]

# Verify that missing values are handled
print("\nAfter handling missing values:")
missing_values_after_handling = combined_data.isnull().sum()
print("Missing values:")
print(missing_values_after_handling)

# Optionally, you can save the modified train and test datasets to new CSV files
train_data.to_csv("train_clean.csv", index=False)
test_data.to_csv("test_clean.csv", index=False)
'''

'\nimport pandas as pd\n\n# Load the training and testing data\n# train_data = pd.read_csv("train.csv")\n# test_data = pd.read_csv("test.csv")\n\n# Concatenate train and test datasets\ncombined_data = pd.concat([train_data, test_data], axis=0)\n\n# Find missing values\nmissing_values = combined_data.isnull().sum()\nprint("Missing values:")\nprint(missing_values)\n\n# Handle missing values (for example, dropping rows with missing values)\ncombined_data.dropna(inplace=True)\n\n# Separate back into train and test datasets\ntrain_data = combined_data.iloc[:len(train_data)]\ntest_data = combined_data.iloc[len(train_data):]\n\n# Verify that missing values are handled\nprint("\nAfter handling missing values:")\nmissing_values_after_handling = combined_data.isnull().sum()\nprint("Missing values:")\nprint(missing_values_after_handling)\n\n# Optionally, you can save the modified train and test datasets to new CSV files\ntrain_data.to_csv("train_clean.csv", index=False)\ntest_data.to_csv("test_cl

In [4]:
'''
After handling missing values, preprocessing should be the next step.
Find a way to preprocess datasets before training classifier.
'''
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Load the clean training and testing data
# train_data = pd.read_csv("train_clean.csv")
# test_data = pd.read_csv("test_clean.csv")

'''
# Reshape and normalize the data
X_train = train_data.drop(columns=["label"]).values.reshape(-1, 28, 28, 1).astype(np.float32) / 255.0
X_test = test_data.values.reshape(-1, 28, 28, 1).astype(np.float32) / 255.0


생략
# Label encoding
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data["label"])
# If you want to convert numerical labels to one-hot encoded vectors, you can use:
'''

# 개발자가 임의로 추가
train_data = train_data.values.reshape(-1,28,28,1)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_data, label, test_size=0.2, random_state=42)

2024-11-11 12:13:36.353685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-11 12:13:36.807806: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# Verify the shapes of the preprocessed data
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", test_data.shape)

X_train shape: (33600, 28, 28, 1)
y_train shape: (33600,)
X_val shape: (8400, 28, 28, 1)
y_val shape: (8400,)
X_test shape: (28000, 784)


In [36]:
'''
In this step, We can build classifier of MNIST dataset.
For this step, we have to find algorithms for building classification.
Find classification algorithms of datasets we dealt, and create a python code.
'''
import tensorflow as tf
from tensorflow.keras import layers, models

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)              

In [38]:
'''
As Final step, we need to prediction file of classifier we built before.
Train classifier we built, and make a prediction file of classifier for submit kaggle competition.
'''
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

''' 
# Load the MNIST dataset
(X_train, y_train), (X_test, _) = mnist.load_data()

# Reshape and normalize the data
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
'''
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val))

# Make predictions on the test data
predictions = model.predict(X_val)
predicted_labels = np.argmax(predictions, axis=1)

# Prepare the submission file
submission_df = pd.DataFrame({
    "ImageId": range(1, len(predicted_labels) + 1),
    "Label": predicted_labels
})

# Save the submission file
submission_df.to_csv("submission_prompt.csv", index=False)

# kaggle score : 0.9863

Epoch 1/5
525/525 [==============================] - 4s 8ms/step - loss: 0.0234 - accuracy: 0.9927 - val_loss: 0.0451 - val_accuracy: 0.9845
Epoch 2/5
525/525 [==============================] - 4s 8ms/step - loss: 0.0201 - accuracy: 0.9936 - val_loss: 0.0422 - val_accuracy: 0.9875
Epoch 3/5
525/525 [==============================] - 4s 8ms/step - loss: 0.0175 - accuracy: 0.9942 - val_loss: 0.0502 - val_accuracy: 0.9857
Epoch 4/5
525/525 [==============================] - 4s 8ms/step - loss: 0.0148 - accuracy: 0.9949 - val_loss: 0.0336 - val_accuracy: 0.9893
Epoch 5/5
263/263 [==============================] - 0s 2ms/step


In [42]:
from sklearn.metrics import accuracy_score
predict = model.predict(X_val)
predicted_labels = np.argmax(predict, axis=1)
acc = round(accuracy_score(y_val,predicted_labels)*100, 2)
acc
# test accuracy : 0.9881

263/263 [==============================] - 0s 2ms/step


99.02

In [43]:
model.save('./gpt_cnn.h5')

/home/dummy/anaconda3/envs/python/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
sub = pd.read_csv('./submission_prompt.csv')

In [44]:
'''
According to the above answer, use hyperparameter tuning technique for improving performance of classifier based on CNN algorithm. 
Find python tools for hyperparameter tuning with example code.
'''
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist

'''
# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess the data
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
'''

# Define the objective function for Optuna
def objective(trial):
    # Define the model architecture
    model = Sequential([
        Conv2D(filters=trial.suggest_categorical('filters', [32, 64]),
               kernel_size=(3, 3),
               activation='relu',
               input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=trial.suggest_categorical('filters', [32, 64]),
               kernel_size=(3, 3),
               activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=trial.suggest_categorical('filters', [32, 64]),
               kernel_size=(3, 3),
               activation='relu'),
        Flatten(),
        Dense(units=trial.suggest_categorical('units', [64, 128]),
              activation='relu'),
        #Dropout(rate=trial.suggest_uniform('dropout_rate', 0.2, 0.5)),
        Dense(units=10, activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=0, )
    
    # Evaluate the model on the test set
    _, accuracy = model.evaluate(X_val, y_val)
    
    return accuracy

# Perform hyperparameter optimization with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the results
print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial
print('  Value: {:.5f}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2024-11-08 17:43:03,275] A new study created in memory with name: no-name-dadae22a-b635-49e8-a364-5fee6115c6a2
/tmp/ipykernel_26790/2503407502.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  model.compile(optimizer=Adam(learning_rate=trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)),


263/263 [==============================] - 1s 2ms/step - loss: 0.2612 - accuracy: 0.9232


[I 2024-11-08 17:43:34,034] Trial 0 finished with value: 0.9232142567634583 and parameters: {'filters': 64, 'units': 128, 'learning_rate': 1.5253026505031117e-05}. Best is trial 0 with value: 0.9232142567634583.


263/263 [==============================] - 1s 2ms/step - loss: 0.0378 - accuracy: 0.9880


[I 2024-11-08 17:44:04,281] Trial 1 finished with value: 0.9879761934280396 and parameters: {'filters': 64, 'units': 128, 'learning_rate': 0.0020803865871640553}. Best is trial 1 with value: 0.9879761934280396.


...

263/263 [==============================] - 1s 2ms/step - loss: 0.0396 - accuracy: 0.9879


[I 2024-11-08 18:05:47,497] Trial 48 finished with value: 0.9878571629524231 and parameters: {'filters': 64, 'units': 64, 'learning_rate': 0.0038187270016301826}. Best is trial 31 with value: 0.9909523725509644.


263/263 [==============================] - 1s 2ms/step - loss: 0.0358 - accuracy: 0.9890


[I 2024-11-08 18:06:19,223] Trial 49 finished with value: 0.989047646522522 and parameters: {'filters': 64, 'units': 128, 'learning_rate': 0.0022632580675899508}. Best is trial 31 with value: 0.9909523725509644.


Number of finished trials: 50
Best trial:
  Value: 0.99095
  Params: 
    filters: 64
    units: 128
    learning_rate: 0.0010474073107695045


In [48]:
best_params = study.best_params

In [49]:
best_params

{'filters': 64, 'units': 128, 'learning_rate': 0.0010474073107695045}

### Best Parameter
Number of finished trials: 50
Best trial:
  Value: 0.99170
  Params: 
    filters: 64
    units: 64
    learning_rate: 0.0021886389727523246

In [30]:
best_params = study.best_params

best_model = Sequential([
        Conv2D(filters=best_params['filters'],
               kernel_size=(3, 3),
               activation='relu',
               input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=study.best_params['filters'],
               kernel_size=(3, 3),
               activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(units=best_params['units'],
              activation='relu'),
        #Dropout(rate=trial.suggest_uniform('dropout_rate', 0.2, 0.5)),
        Dense(units=10, activation='softmax')
    ])
best_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [31]:
best_model.fit(X_train, y_train, epochs=10)
best_model.evaluate(X_test, y_test)
# test accuracy : 0.9883

Epoch 1/10
1050/1050 [==============================] - 10s 9ms/step - loss: 0.1674 - accuracy: 0.9489
Epoch 2/10
1050/1050 [==============================] - 10s 9ms/step - loss: 0.0516 - accuracy: 0.9839
Epoch 3/10
1050/1050 [==============================] - 10s 9ms/step - loss: 0.0366 - accuracy: 0.9889
Epoch 4/10
1050/1050 [==============================] - 10s 10ms/step - loss: 0.0270 - accuracy: 0.9912
Epoch 5/10
1050/1050 [==============================] - 10s 10ms/step - loss: 0.0176 - accuracy: 0.9944
Epoch 6/10
1050/1050 [==============================] - 10s 9ms/step - loss: 0.0141 - accuracy: 0.9953
Epoch 7/10
1050/1050 [==============================] - 12s 11ms/step - loss: 0.0138 - accuracy: 0.9955
Epoch 8/10
1050/1050 [==============================] - 9s 8ms/step - loss: 0.0097 - accuracy: 0.9968
Epoch 9/10
1050/1050 [==============================] - 11s 10ms/step - loss: 0.0091 - accuracy: 0.9968
Epoch 10/10
263/263 [==============================] - 1s 2ms/step - l

[0.03631014749407768, 0.9898809790611267]

In [34]:
best_model.save('./gpt_cnn_HP.h5')

/home/dummy/anaconda3/envs/python/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
sub = pd.read_csv('./sample_submission.csv')
sub

FileNotFoundError: [Errno 2] No such file or directory: './sample_submission.csv'

In [33]:
y_pred = best_model.predict(X_test)
submission_df = pd.DataFrame({
    "ImageId": sub['ImageId'],
    "Label": y_pred.argmax(axis = 1)
})
submission_df.to_csv('submission_gpt_optimization.csv', index= False)
# kaggle score : 0.996

263/263 [==============================] - 1s 2ms/step


NameError: name 'sub' is not defined

In [45]:
len(y_pred)

28000

In [23]:
y_pred.argmax(axis = 1)

array([7, 2, 1, ..., 4, 5, 6])

In [31]:
test = pd.read_csv('./submission_gpt_optimization.csv')
test

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
